In [7]:
import pandas as pd
import csv
from datetime import datetime
from datetime import time

df = pd.read_csv('TNP-trips.csv') #reading in csv file
df = df.loc[:, ['Trip Start Timestamp', 'Trip Seconds', 'Trip Miles', 'Fare', 'Tip', 'Trip Total']] #selecting relevant columns
all_tip_df = df[df['Tip'] > 0] # creating separate dataframe that only includes tipped rides

time = []
tipfare = []
tipmile = []

time2 = []

for i in range(len(df)):
    timeslot = (df.iloc[i,0])
    tip = float(df.iloc[i,4])
    miles = float(df.iloc[i,2])
    fare = float(df.iloc[i,3])
    if fare > 0:
        tipfare.append(tip/fare)
    else:
        tipfare.append(0)
    if miles > 0:
        tipmile.append(tip/miles)
    else:
        tipmile.append(0)
    time.append(datetime.strptime(timeslot, "%m/%d/%Y %H:%M").time())
    
for i in range(len(all_tip_df)):
    timeslot = (all_tip_df.iloc[i,0])
    time2.append(datetime.strptime(timeslot, "%m/%d/%Y %H:%M").time())

df['Time'] = time
df['Tip/Fare'] = tipfare
df['Tip/Mile'] = tipmile

all_tip_df['Time'] = time2

df = df.groupby(['Time']).mean()
all_tip_df = all_tip_df.groupby(['Time']).mean()

tiprate = []
nontiprate = []
for i in range(len(df)):
    expectedTip = float(df.iloc[i,2])
    nonzeroTip = float(all_tip_df.iloc[i,2])
    percent = (expectedTip/nonzeroTip) * 100
    tiprate.append(percent)
    nontiprate.append(100 - percent)
df['Tip Rate'] = tiprate
df['Non-Tip Rate'] = nontiprate

df = df.reset_index()
all_tip_df = all_tip_df.reset_index()

C:\Users\Carso\AppData\Local\Temp\ipykernel_19964\988981234.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
import plotly.express as px

def graph(Selection):
    if Selection == 'Tip Rate':
        fig = px.line(df, x='Time', y = 'Tip Rate', labels={'Tip Rate':'Tip Rate (%)', 'Time':'Time of Day'})
        fig.show()
    if Selection == 'Average Non-Zero Tip':
        fig = px.line(all_tip_df, x='Time', y='Tip', labels={'Tip':'Average Non-Zero Tip ($)', 'Time':'Time of Day'})
        fig.show()
    if Selection == 'Average Expected Tip':
        fig = px.line(df, x='Time', y = 'Tip', labels={'Tip':'Average Expected Tip ($)', 'Time':'Time of Day'})
        fig.show()
        
def ratioplot(trend, Add_Trendline):
    if trend == 'Fare':
        if Add_Trendline == 'Yes':
            fig3 = px.scatter(df, x='Fare', y='Tip', trendline="ols")
        else:
            fig3 = px.scatter(df, x='Fare', y='Tip')
        fig3.show()
    if trend == 'Miles':
        if Add_Trendline == 'Yes':
            fig3 = px.scatter(df, x='Trip Miles', y='Tip', trendline="ols")
        else:
            fig3 = px.scatter(df, x='Trip Miles', y='Tip')
        fig3.show()

def recommend(time):
    global df
    df = df.groupby(['Time']).mean()
    time = datetime.strptime(time, "%H:%M").time()
    avg = 0.4867473
    current = (df.loc[time])['Tip']
    if current / avg >= 1.5:
        sub1 = '<font color="green">Great!!</font>'
        sub2 = '{}% higher than'.format(int((current/avg)*100) - 100)
    elif current / avg > 1:
        sub1 = '<font color="green">Good</font>'
        sub2 = '{}% higher than'.format(int((current/avg)*100) - 100)
    elif current / avg == 1:
        sub1 = '<font color="yellow">Neutral</font>'
        sub2 = 'equal to'
    elif current / avg <= 0.5:
        sub1 = '<font color="red">BAD</font>'
        sub2 = '{}% lower than'.format(100 - int((current/avg)*100))
    else:
        sub1 = '<font color="red">Poor</font>'
        sub2 = '{}% lower than'.format(100 - int((current/avg)*100))
    html_string3 = '<font size="3">The projection for this time of day is:</font> <font size="4"><b>{}</b></font>'.format(sub1)
    html_string4 = '<font size="3">You can expect a tip of \${:.2f}, which is {} the average of \$0.49</font>'.format(current, sub2)
    display(HTML(html_string3))
    display(HTML(html_string4))
    df = df.reset_index()

In [11]:
from IPython.display import HTML, display
from ipywidgets import interact

html_string = '<h1><center><u>{}</u></center></h1>'.format('Rideshare Tipping in the City of Chicago')
display(HTML(html_string))
html_string2 = '<center><body style="font-size:450px;">You can access the data source <body style="color:#FFFF00;"><a href=https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p>here</a></body>.</body></center>'
display(HTML(html_string2))
print()

print('This visualization depicts a variety of tipping metrics and their trends throughout the day.')
print('Select a metric to visualize:')
interact(graph, Selection=['Tip Rate', 'Average Non-Zero Tip', 'Average Expected Tip'])
print()

print('Now explore the relationship between tips and ride fare or miles traveled')
print('Pick a trend to explore:')
interact(ratioplot, trend=['Fare', 'Miles'], Add_Trendline=['Yes', 'No'])
print()

html_string5 = '<h3><center><u>{}</u></center></h3>'.format('Driver Tool')
display(HTML(html_string5))
print('This tool will help you decide whether it is an optimal time to be driving or if you would be better served waiting a bit.')
print('Enter the time of day:')
interact(recommend, time='00:00')
print()


This visualization depicts a variety of tipping metrics and their trends throughout the day.
Select a metric to visualize:


interactive(children=(Dropdown(description='Selection', options=('Tip Rate', 'Average Non-Zero Tip', 'Average …


Now explore the relationship between tips and ride fare or miles traveled
Pick a trend to explore:


interactive(children=(Dropdown(description='trend', options=('Fare', 'Miles'), value='Fare'), Dropdown(descrip…

This tool will help you decide whether it is an optimal time to be driving or if you would be better served waiting a bit.
Enter the time of day:


interactive(children=(Text(value='00:00', description='time'), Output()), _dom_classes=('widget-interact',))